## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

### 使用文件加载专用chains（官方弃用）
### Use of file-loading-specific chains (officially deprecated)
*****

In [2]:
#from langchain.chains import load_chain
#chain = load_chain("./example2/chain.json")
#print(chain.run("2+6等于几?"))
#chain = load_chain("./example2/chain2.json")
#chain.run("男人")

- Langchain已经启用这种chains加载方式，而改用自己的hub:https://smith.langchain.com/hub
- Langchain has enabled this way of loading chains in favor of using its own hub:https://smith.langchain.com/hub
- 代码见：https://github.com/langchain-ai/langchain/blob/b34f1086fe723c283e48bca63e7917600152b263/libs/langchain/langchain/chains/loading.py#L628C50-L628C81
- code at:https://github.com/langchain-ai/langchain/blob/b34f1086fe723c283e48bca63e7917600152b263/libs/langchain/langchain/chains/loading.py#L628C50-L628C81

### CustomChain
- 当通用链不满足的时候，可以自行构建来实现特定的目的
- When the generic chain is not satisfied, you can build your own to fulfill a specific purpose
<hr>

In [3]:
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import (
    CallbackManagerForChainRun
)
from langchain.chains.base import  Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import  BaseLanguageModel


In [4]:
class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt:BasePromptTemplate
    llm:BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) -> List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables
    
    @property
    def output_keys(self) -> List[str]:
        """将始终返回text键"""
        return [self.out_key]
    
    def _call(
        self,
        inputs:Dict[str,Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    ) -> Dict[str,Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        #print("prompt_value:",prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        #print("response:",response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"
       


In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import  PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(
        temperature=0
    ),
)

In [6]:
result = chain.invoke({"topic":"降本增效"})
print(result)

{'topic': '降本增效', 'text': '降本增效（英文：Cost reduction and efficiency improvement）是指企业或组织通过优化资源配置和管理，降低成本，提高生产效率和经营效益的一种管理方法。降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。\n\n降本增效的目标是通过降低生产成本和提高生产效率，实现企业的盈利最大化。在实施降本增效的过程中，企业需要进行全面的成本分析，找出造成成本增加的原因，并采取相应的措施进行改进。这些措施可以包括优化生产流程、提高员工效率、降低原材料成本、减少浪费等。\n\n降本增效的实施需要全员参与，需要企业管理层和员工共同努力。管理层需要制定明确的降本增效目标和计划，为员工提供必要的培训和支持，确保计划的顺利实施。员工则需要积极配合，提高工作效率，减少浪费，共同为企业的发展贡献力量。\n\n降本增效不仅可以帮助企业提高盈利能力，还可以提高企业的竞争力，为企业创造更多的发展机会。因此，企业在日常经营管理中应该重视降本增效工作，不断优化管理模式，提高生产效率，实现经济效益和社会效益的双赢。\n\n总之，降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。企业应该重视降本增效工作，不断优化管理模式，提高生产效率，实现经济效益和社会效益的双赢。'}
